<a href="https://colab.research.google.com/github/halimatu001/Data_Science_Assignment/blob/main/Week_5_ADAP_FE_23_83785742_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import sqlite3
# Set display options to show all columns
pd.set_option('display.max_columns', None)


In [ ]:
path ="/content/drive/MyDrive/3MTT Deeptech /job_change_data.csv"
df = pd.read_csv(path)
df.head(5)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,32403,city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21
1,9858,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98
2,31806,city_21,0.624,Male,No relevent experience,no_enrollment,High School,NaN,<1,NaN,Pvt Ltd,never,15
3,27385,city_13,0.827,Male,Has relevent experience,no_enrollment,Masters,STEM,11,Oct-49,Pvt Ltd,1,39
4,27724,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,>4,72


In [ ]:
# Define a mapping for company_size to numeric levels
size_map = {
    '<10': 1,
    '10/49': 2,
    '50-99': 3,
    '100-500': 4,
    '500-999': 5,
    '1000-4999': 6,
    '5000-9999': 7,
    '10000+': 8
}


In [ ]:
# Apply the mapping (non-matching values like 'Oct-49' will become NaN)
df['company_size_num'] = df['company_size'].map(size_map)

In [ ]:
# Filter rows where city_development_index > 0.8 and company_size > 3
filtered_df = df[(df['city_development_index'] > 0.8) & (df['company_size_num'] > 3)]


In [ ]:
# Show result
print(filtered_df[['enrollee_id', 'city_development_index', 'company_size']].head())


    enrollee_id  city_development_index company_size
4         27724                   0.920       10000+
13        24372                   0.949      100-500
21        16001                   0.920       10000+
36        22139                   0.920      100-500
37        19644                   0.913       10000+


Use iloc to select the first 10 rows and specific columns like Experience and
Education_Level.

In [ ]:
# Use iloc to select first 10 rows and columns for experience and education_level
selected_data = df.iloc[:10, [6, 8]]

# Display the result
print(selected_data)


  education_level experience
0        Graduate          9
1        Graduate          5
2     High School         <1
3         Masters         11
4        Graduate        >20
5         Masters         10
6        Graduate         <1
7        Graduate        >20
8        Graduate         14
9        Graduate          3


Group data by Relevant_Experience and calculate the average
City_Development_Index for each group.

In [ ]:
# Group by 'relevent_experience' and calculate the mean of 'city_development_index'
grouped_avg = df.groupby('relevent_experience')['city_development_index'].mean()

# Display the result
print(grouped_avg)

relevent_experience
Has relevent experience    0.831000
No relevent experience     0.809828
Name: city_development_index, dtype: float64


Group by Company_Size and count the number of unique entries in
Last_New_Job.

In [ ]:
# Group by 'company_size' and count unique 'last_new_job' entries
unique_counts = df.groupby('company_size')['last_new_job'].nunique()

# Display the result
print(unique_counts)

company_size
100-500      6
1000-4999    6
10000+       6
50-99        6
500-999      6
5000-9999    6
<10          6
Oct-49       6
Name: last_new_job, dtype: int64


Analyze the frequency distribution of Company_Type using value_counts().

In [ ]:
# Analyze frequency distribution of 'company_type'
company_type_distribution = df['company_type'].value_counts()

# Display the result
print(company_type_distribution)

company_type
Pvt Ltd                1141
Public Sector           127
Funded Startup           97
Early Stage Startup      65
NGO                      53
Other                    12
Name: count, dtype: int64


Identify numerical columns with missing values and fill them using the mean

In [ ]:
# Select only numerical columns
numerical_cols = df.select_dtypes(include=['number'])

# Identify numerical columns with missing values
missing_numerical = numerical_cols.columns[numerical_cols.isnull().any()]

# Fill missing values in those columns with the mean
for col in missing_numerical:
    mean_value = df[col].mean()
    df[col].fillna(mean_value, inplace=True)

# Optional: display filled columns and their means
for col in missing_numerical:
    print(f"Filled missing values in '{col}' with mean: {df[col].mean():.2f}")

Drop rows where more than 50% of data is missing, and document the impact
on dataset size.

In [ ]:
# Record original size
original_shape = df.shape

# Drop rows with more than 50% missing values
threshold = df.shape[1] / 2  # 50% of total columns
df_cleaned = df.dropna(thresh=threshold)

# Record new size
new_shape = df_cleaned.shape

# Document the impact
print(f"Original dataset size: {original_shape[0]} rows, {original_shape[1]} columns")
print(f"New dataset size:      {new_shape[0]} rows, {new_shape[1]} columns")
print(f"Rows dropped:          {original_shape[0] - new_shape[0]}")

Original dataset size: 2129 rows, 14 columns
New dataset size:      2129 rows, 14 columns
Rows dropped:          0


Query the dataset to extract rows where Experience > 10 and Company_Size ==
7.

In [ ]:
# Step 1: Map experience to numeric
experience_map = {
    '<1': 0,
    '1': 1, '2': 2, '3': 3, '4': 4,
    '5': 5, '6': 6, '7': 7, '8': 8,
    '9': 9, '10': 10,
    '11': 11, '12': 12, '13': 13, '14': 14,
    '15': 15, '16': 16, '17': 17, '18': 18, '19': 19,
    '20': 20, '>20': 21
}
df['experience_num'] = df['experience'].replace(experience_map).astype('float')

# Step 2: Map company_size to numeric scale
size_map = {
    '<10': 1,
    '10/49': 2,
    '50-99': 3,
    '100-500': 4,
    '500-999': 5,
    '1000-4999': 6,
    '5000-9999': 7,
    '10000+': 8
}
df['company_size_num'] = df['company_size'].map(size_map)

# Step 3: Query the dataset
filtered_df = df[(df['experience_num'] > 10) & (df['company_size_num'] == 7)]

# Display the result
print(filtered_df[['experience', 'company_size', 'experience_num', 'company_size_num']].head())


    experience company_size  experience_num  company_size_num
112        >20    5000-9999            21.0               7.0
127        >20    5000-9999            21.0               7.0
154         13    5000-9999            13.0               7.0
158        >20    5000-9999            21.0               7.0
283         11    5000-9999            11.0               7.0


<ipython-input-19-33178555>:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['experience_num'] = df['experience'].replace(experience_map).astype('float')


Create a new feature Experience_Gap by subtracting Last_New_Job from
Experience.

In [ ]:
# Map 'last_new_job' to numeric
last_job_map = {
    'never': 0,
    '<1': 0.5,
    '1': 1, '2': 2, '3': 3, '4': 4, '>4': 5
}
df['last_new_job_num'] = df['last_new_job'].replace(last_job_map).astype('float')

# Step 3: Create the new feature
df['experience_gap'] = df['experience_num'] - df['last_new_job_num']

# Display the new feature
print(df[['experience', 'last_new_job', 'experience_num', 'last_new_job_num', 'experience_gap']].head())


  experience last_new_job  experience_num  last_new_job_num  experience_gap
0          9            1             9.0               1.0             8.0
1          5            1             5.0               1.0             4.0
2         <1        never             0.0               0.0             0.0
3         11            1            11.0               1.0            10.0
4        >20           >4            21.0               5.0            16.0


<ipython-input-20-1540036628>:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['last_new_job_num'] = df['last_new_job'].replace(last_job_map).astype('float')


Normalize City_Development_Index to a 0–1 scale and explain the benefits of
normalization.

In [ ]:
# Apply Min-Max normalization
df['city_development_index_norm'] = (df['city_development_index'] - df['city_development_index'].min()) / \
                                     (df['city_development_index'].max() - df['city_development_index'].min())

# Display normalized values
print(df[['city_development_index', 'city_development_index_norm']].head())


   city_development_index  city_development_index_norm
0                   0.827                     0.756487
1                   0.920                     0.942116
2                   0.624                     0.351297
3                   0.827                     0.756487
4                   0.920                     0.942116


Create new column cdi_per and merge it to the original dataframe using
pd.merge and analyze the combined insights.

In [ ]:
# Create a DataFrame with enrollee_id and cdi_per (City Development Index %)
df_cdi = df[['enrollee_id', 'city_development_index']].copy()
df_cdi['cdi_per'] = df_cdi['city_development_index'] * 100

# Merge this new column back into the original dataframe using enrollee_id
df_merged = pd.merge(df, df_cdi[['enrollee_id', 'cdi_per']], on='enrollee_id', how='left')

# Display the result
print(df_merged[['enrollee_id', 'city_development_index', 'cdi_per']].head())


   enrollee_id  city_development_index  cdi_per
0        32403                   0.827     82.7
1         9858                   0.920     92.0
2        31806                   0.624     62.4
3        27385                   0.827     82.7
4        27724                   0.920     92.0
